In [1]:
import pandas as pd
import numpy as np

In [2]:
trn = pd.read_csv('./data/train_ver2.csv')

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
prods = trn.columns[24:].tolist()

In [4]:
prods

['ind_ahor_fin_ult1',
 'ind_aval_fin_ult1',
 'ind_cco_fin_ult1',
 'ind_cder_fin_ult1',
 'ind_cno_fin_ult1',
 'ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1',
 'ind_ctpp_fin_ult1',
 'ind_deco_fin_ult1',
 'ind_deme_fin_ult1',
 'ind_dela_fin_ult1',
 'ind_ecue_fin_ult1',
 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1',
 'ind_plan_fin_ult1',
 'ind_pres_fin_ult1',
 'ind_reca_fin_ult1',
 'ind_tjcr_fin_ult1',
 'ind_valo_fin_ult1',
 'ind_viv_fin_ult1',
 'ind_nomina_ult1',
 'ind_nom_pens_ult1',
 'ind_recibo_ult1']

In [5]:
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")]
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

In [6]:
trn['int_date'] = trn['fecha_dato'].map(date_to_int).astype(np.int8)

In [7]:
trn_lag = trn.copy()
trn_lag['int_date'] += 1
trn_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in trn.columns]

In [8]:
trn_lag.columns

Index(['fecha_dato_prev', 'ncodpers', 'ind_empleado_prev',
       'pais_residencia_prev', 'sexo_prev', 'age_prev', 'fecha_alta_prev',
       'ind_nuevo_prev', 'antiguedad_prev', 'indrel_prev',
       'ult_fec_cli_1t_prev', 'indrel_1mes_prev', 'tiprel_1mes_prev',
       'indresi_prev', 'indext_prev', 'conyuemp_prev', 'canal_entrada_prev',
       'indfall_prev', 'tipodom_prev', 'cod_prov_prev', 'nomprov_prev',
       'ind_actividad_cliente_prev', 'renta_prev', 'segmento_prev',
       'ind_ahor_fin_ult1_prev', 'ind_aval_fin_ult1_prev',
       'ind_cco_fin_ult1_prev', 'ind_cder_fin_ult1_prev',
       'ind_cno_fin_ult1_prev', 'ind_ctju_fin_ult1_prev',
       'ind_ctma_fin_ult1_prev', 'ind_ctop_fin_ult1_prev',
       'ind_ctpp_fin_ult1_prev', 'ind_deco_fin_ult1_prev',
       'ind_deme_fin_ult1_prev', 'ind_dela_fin_ult1_prev',
       'ind_ecue_fin_ult1_prev', 'ind_fond_fin_ult1_prev',
       'ind_hip_fin_ult1_prev', 'ind_plan_fin_ult1_prev',
       'ind_pres_fin_ult1_prev', 'ind_reca_fin_ult1

In [9]:
df_trn = trn.merge(trn_lag, on=['ncodpers', 'int_date'], how='left')

In [10]:
del trn, trn_lag

In [11]:
# fill in null with, first month data have no prev data.
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)

In [12]:
# creating padd as new purchasing data, 
# it meas current month(df_trn[prod]) has product but previous month(df_trn[prod + '_prev']) has not
for prod in prods:
    padd = prod + '_add'
    prev = prod + '_prev'
    df_trn[padd] = ((df_trn[prod] == 1) & (df_trn[prev] == 0)).astype(np.int8)

In [13]:
# filter only new purchasing data and save it.
add_cols = [prod + '_add' for prod in prods]
labels = df_trn[add_cols].copy()
labels.columns = prods
labels.to_csv('./data/labels.csv', index=False)